### 简介
当前的关键词抽取方法有三种：
* TF-IDF：基于文本的词频和逆文本频率来考量文本对于一篇文章或一句话的重要程度

* TextRank：其基于PageRank演化过来，基于一种邻接图思想来表征一个分词结果的重要性

* LDA：隐含狄利克雷分布，相当于一种生成方法，对当前分词结果的主题分布进行建模

本篇猪皮的主要任务是，使用TsingHua中刘知远教授在论文：Automatic Keyphrase Extraction by Bridging Vocabulary Gap中提供的网易新闻数据集，是做一个中文的关键词抽取demo，这里只取数据集中的content和tags。

In [1]:
import jieba.analyse
import json
import codecs
import jieba

In [2]:
file_path = "data/163_chinese_news_dataset_2011.dat"
content = []  # 存放读取出来的content，之后要做文本预处理
tags = []  # 存放每一个content对应的标签，这里简化了任务，只使用了content，其实这个tags应该也与summary和title字段都有关
with codecs.open(file_path, 'r', 'utf-8') as fr:
    for line in fr.readlines():
        d = json.loads(line)
        cont = d['content']
        tag = [w for w in set(d['tags'])]  # 这里关键词可能有多个，看规格，最后在从jieba那里的参数设定选取几个关键词即可
        content.append(cont)
        tags.append(tag)
print(content[:3])
print(tags[:3])

['重庆晚报6月11日报道  昨日，市政府公众信息网发布消息称，经2010年5月13日市政府第70次常务会议通过，给予文强、陈洪刚二人行政开除处分。\n今年4月14日，市第五中级人民法院以受贿罪，包庇、纵容黑社会性质组织罪，巨额财产来源不明罪，强奸罪数罪并罚判处文强死刑，剥夺政治权利终身，并处没收个人全部财产。5月21日，市高级人民法院对文强案二审宣判，依法驳回文强上诉，维持一审的死刑判决。\n2月25日，市公安局交警总队原总队长陈洪刚受贿案在市第五中级人民法院一审宣判。陈洪刚因犯受贿，包庇、纵容黑社会性质组织，巨额财产来源不明，伪造居民身份证罪，数罪并罚，被判处有期徒刑20年，没收个人财产40万元人民币，追缴赃款326万余元及不明来源财产584万余元。记者 李伟\n', '6月11日，伊朗总统艾哈迈迪・内贾德在国家馆日活动上致辞。\n6月11日，伊朗总统艾哈迈迪・内贾德（右）在伊朗代表团举行的新闻发布会上。\n环球时报6月12日报道 据6月12日出版的《环球时报》报道，伊朗总统内贾德11日在中国上海参加世博会伊朗馆日活动。由于联合国安理会9日刚通过制裁伊朗决议，内贾德此时访华非常受外界关注。\n美联社11日称，内贾德在上海依然“声讨”美国。报道援引内贾德出席伊朗馆日活动时的话称：“美国人有什么权利对希望和平利用核能的国家施压？别忘了美国是唯一一个对别国使用核武器的国家。”\n路透社11日在题为“伊朗总统出席世博会，高度评价中伊关系”的文章中称，内贾德对中伊两国关系“高度赞扬”。报道称，联合国安理会9日通过进一步制裁伊朗的决议，中国投了赞成票，但内贾德对此似乎不以为意，他的矛头显然针对西方国家。\n内贾德11日表示，安理会对伊朗的新制裁“没有作用”。伊朗议会国安与外交委员会主席鲍洛哲迪也强硬表示，伊朗将展开对国际原子能机构的反制行动，包括伊朗议会最快在13日开会，讨论推动以立法限制国际原子能机构检查行动的可能性。\n对于西方媒体纷纷猜测“内贾德访华是否会见中国官员”，中国外交部发言人秦刚在外交部例行记者会上表示，内贾德总统来中国只参加和上海世博会有关的活动。\n', '人民网6月12日报道 昨天，住建部政策研究中心副主任王珏林表示，中央已经对此次楼市调控有可能带来的影响做好了充分准备，调控政策不会半途而废。\n王珏林说，目前包括部分地方政府在内，各方主体对此次楼市调

In [25]:
# 在这里，jieba有封装好extract_tags方法，可对文本进行分词、关键词抽取等一系列操作
from sklearn import metrics
from sklearn.preprocessing import MultiLabelBinarizer

def extract_tfidf(string, tags, topK, allowPOS):
    """
    string:待测字符串列表
    tags:真实标签列表
    topK:选取得分最高的K个关键字
    allowPOS:是否针对分词的词性进行处理，指定何种词性
    """
    jieba.analyse.set_stop_words("data/stopWord.txt") # 加载自定义停用词表
    
    pred_keyword = []  # 存放模型输出的keyword
    for s in string:
        pred = jieba.analyse.extract_tags(s, topK=topK, allowPOS=allowPOS)  # 在这里extract_tags默认方法是tf-idf的
        pred_keyword.append(pred)
    
#     MB = MultiLabelBinarizer()
    
#     y_tags = MB.fit_transform(tags)
#     y_pred = MB.transform(pred_keyword)
    
    # prf = metrics.precision_recall_fscore_support(tags, pred_keyword)  # 在sklearn0.17版本以后，不允许直接使用多标签
#     prf = metrics.precision_recall_fscore_support(y_tags, y_pred)
    
    return pred_keyword

In [29]:
def extract_textRank(string, tags, span, topK, allowPOS):
    """
    string:待测字符串列表
    tags:真实标签列表
    span:textRank对应的邻接窗口大小
    topK:选取得分最高的K个关键字
    allowPOS:是否针对分词的词性进行处理，指定何种词性
    """
    jieba.analyse.set_stop_words("data/stopWord.txt") # 加载自定义停用词表
    
    pred_keyword = []
    for s in string:
        trank = jieba.analyse.TextRank()
        trank.span = span
        pred = trank.textrank(s, topK=topK, allowPOS=allowPOS)
        pred_keyword.append(pred)
        
#     MB = MultiLabelBinarizer()
        
#     y_tags = MB.fit_transform(tags)
#     y_pred = MB.transform(pred_keyword)
    
    # prf = metrics.precision_recall_fscore_support(tags, pred_keyword)  # 在sklearn0.17版本以后，不允许直接使用多标签
#     prf = metrics.precision_recall_fscore_support(y_tags, y_pred)
    
    return pred_keyword

In [30]:
import time

key_word_pos = ['x', 'ns', 'n', 'vn', 'v', 'l', 'j', 'nr', 'nrt', 'nt', 'nz', 'nrfg', 'm', 'i',
            'an', 'f', 't', 'b', 'a', 'd', 'q', 's', 'z']

t0 = time.time()

tfidf = extract_tfidf(content, tags, topK=2, allowPOS=key_word_pos)

print("tf-idf time use: {}".format(time.time()-t0))

t1 = time.time()
textRank1 = extract_textRank(content, tags, span=5, topK=2, allowPOS=key_word_pos)

print("textRank time use: {}".format(time.time()-t1))

tf-idf time use: 492.06223154067993
textRank time use: 548.3995642662048


In [31]:
for i in range(10):
    print(tags[i])
    print(tfidf[i])
    print(textRank1[i])
    print("-----------------")

['文强', '重庆']
['文强', '陈洪刚']
['文强', '陈洪刚']
-----------------
['世博', '制裁', '伊朗']
['内贾德', '伊朗']
['内贾德', '伊朗']
-----------------
['调控', '楼市']
['调控', '王珏林']
['调控', '楼市']
-----------------
['养生', '健康素养']
['卫生部', '健康']
['健康', '卫生部']
-----------------
['搬迁', '景区']
['石林', '村民']
['石林', '村民']
-----------------
['工人', '恐怖袭击']
['阿尔及利亚', '维拉']
['阿尔及利亚', '武装']
-----------------
['警察', '打砸', '上访']
['茂名', '石化']
['石化', '茂名']
-----------------
['富户密集度', '富豪']
['富户', '百万美元']
['富户', '全球']
-----------------
['监狱', '死亡', '河北']
['海涛', '监狱']
['海涛', '监狱']
-----------------
['副局长', '被砍', '情感纠葛']
['罗平县', '男子']
['罗平县', '男子']
-----------------


### textrank与TF-IDF
* 1.TextRank与TFIDF均严重依赖于分词结果——如果某词在分词时被切分成了两个词，那么在做关键词提取时无法将两个词黏合在一起（TextRank有部分黏合效果，但需要这两个词均为关键词）。因此是否添加标注关键词进自定义词典，将会造成准确率、召回率大相径庭。
* 2.TextRank的效果并不优于TFIDF。
* 3.TextRank虽然考虑到了词之间的关系，但是仍然倾向于将频繁词作为关键词。
* 4.TextRank涉及到构建词图及迭代计算，所以提取速度较慢。